# Model definition

Let's create a model with 2 Relu with Dropout hidden layers, and a Softmax output layer. Set the number of units in each 
hidden layer to 32, and use dropout rate of 0.2. Moreover, let's use Adam optimizer with `learn_rate = 0.001`, 
10 iterations over batches of 128 elements.


In [1]:
!pip install "thinc>=8.0.0a0" ml_datasets "tqdm>=4.41" "torch"


In [2]:
# Perform operations on GPU if available
from thinc.api import prefer_gpu
prefer_gpu()


False

In [3]:
# Add src to sys.path

import sys
import os
from pathlib import PurePath

# add custom python modules root to the path variable,
root_path = PurePath(os.getcwd()).parents[0]
print(root_path)
src_path = str(root_path.joinpath('src'))

if src_path not in sys.path:
    sys.path.insert(0, str(src_path))

print(sys.path)




/Users/jean.metz/workspace/jmetzz/sandbox-thinc.ai
['/Users/jean.metz/workspace/jmetzz/sandbox-thinc.ai/src', '/Users/jean.metz/workspace/jmetzz/sandbox-thinc.ai/notebooks', '/Users/jean.metz/miniconda/envs/thinc.ai/lib/python37.zip', '/Users/jean.metz/miniconda/envs/thinc.ai/lib/python3.7', '/Users/jean.metz/miniconda/envs/thinc.ai/lib/python3.7/lib-dynload', '', '/Users/jean.metz/miniconda/envs/thinc.ai/lib/python3.7/site-packages', '/Users/jean.metz/miniconda/envs/thinc.ai/lib/python3.7/site-packages/IPython/extensions', '/Users/jean.metz/.ipython']


In [4]:
import torch
import torch.nn
import torch.nn.functional as F
from thinc.api import PyTorchWrapper, Adam

width = 32
nO = 10
nI = 784
dropout = 0.2

class PyTorchModel(torch.nn.Module):
    def __init__(self, width, nO, nI, dropout):
        super(PyTorchModel, self).__init__()
        self.dropout1 = torch.nn.Dropout2d(dropout)
        self.dropout2 = torch.nn.Dropout2d(dropout)
        self.fc1 = torch.nn.Linear(nI, width)
        self.fc2 = torch.nn.Linear(width, nO)

    def forward(self, x):
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    
wrapped_pytorch_model = PyTorchWrapper(PyTorchModel(width, nO, nI, dropout))
wrapped_pytorch_model

In [6]:
# You can use this training code to train the model:
import ml_datasets

(train_X, train_Y), (test_X, test_Y) = ml_datasets.mnist()
optimizer = Adam(0.001)

from train import train_model
train_model(((train_X, train_Y), (test_X, test_Y)), wrapped_pytorch_model, optimizer, n_iter=10, batch_size=128)


 30%|██▉       | 126/422 [00:00<00:00, 624.51it/s]

0	3156285.10	0.931


 15%|█▍        | 63/422 [00:00<00:00, 629.06it/s] 

1	3156250.42	0.930


 30%|███       | 127/422 [00:00<00:00, 633.01it/s]

...

 15%|█▍        | 63/422 [00:00<00:00, 628.22it/s] 

7	3156206.63	0.931


 30%|██▉       | 126/422 [00:00<00:00, 626.09it/s]

8	3156213.67	0.934


9	3156194.79	0.932
